In [1]:
import pandas as pd
import numpy
df = pd.read_csv('BX-Book-Ratings.csv', names=["User-ID", "ISBN", "Book-Rating"], delimiter=";")
rating_matrix = numpy.ndarray(shape=(50000,50000), dtype = 'int64')
df_mat = df.as_matrix()[1:]
#print(df_mat)

/Users/priyadharshenib/anaconda3/envs/cs670/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
%store df_mat

Stored 'df_mat' (ndarray)


In [3]:
%store -r df_mat
totalratings = len(df_mat[:,0])
print totalratings

1149780


In [4]:
print (type(df_mat[:,0][1]))

<type 'str'>


In [5]:
import numpy as np
df_mat[:,0] = df_mat[:,0].astype('int64')
df_mat[:,2] = df_mat[:,2].astype('int64')

usercount = (len(np.unique(df_mat[:,0])))
bookcount = (len(np.unique(df_mat[:,1])))
#numpy.unique(df_mat[:,1]).shape

In [6]:
print (type(df_mat[:,0][2]))

<type 'int'>


In [21]:
LH_dict = dict()
LH_dict1 = dict()
LH_dict2 = dict()

In [8]:
print usercount
print bookcount

105283
340556


In [22]:
bookid = 0
usernum = 0
bookmap = {}
usermap = {}
useridslist = []
for k in range(0,800000):
    userid = df_mat[k,0]
    isbn = df_mat[k,1]
    if isbn not in bookmap:
        if bookid < 10000:
            bookmap[isbn] = bookid
            if userid not in usermap:
                if usernum < 10000:
                    usermap[userid]=usernum
                    rating_matrix[usernum, bookid] = df_mat[k,2]
                    if df_mat[k,2] != 0:
                        if df_mat[k,2] > 7:
                            LH_dict1[(userid, isbn)] = 'L'
                        else:
                            LH_dict1[(userid, isbn)] = 'H'
                        LH_dict2[(userid, isbn)] = df_mat[k,2]
                    usernum+=1
            else:
                rating_matrix[usermap[userid], bookid] = df_mat[k,2]
                if df_mat[k,2] != 0:
                    if df_mat[k,2] > 7:
                        LH_dict1[(userid, isbn)] = 'L'
                    else:
                        LH_dict1[(userid, isbn)] = 'H'
                    LH_dict2[(userid, isbn)] = df_mat[k,2]
            bookid = bookid+1
    else:
        if userid not in usermap:
            if usernum < 10000:
                usermap[userid]=usernum
                rating_matrix[usernum, bookmap[isbn]] = df_mat[k,2]
                if df_mat[k,2] != 0:
                    if df_mat[k,2] > 7:
                        LH_dict1[(userid, isbn)] = 'L'
                    else:
                        LH_dict1[(userid, isbn)] = 'H'
                    LH_dict2[(userid, isbn)] = df_mat[k,2]
                usernum+=1
        else:
            rating_matrix[usermap[userid], bookmap[isbn]] = df_mat[k,2]
            if df_mat[k,2] != 0:
                if df_mat[k,2] > 7:
                    LH_dict1[(userid, isbn)] = 'L'
                else:
                    LH_dict1[(userid, isbn)] = 'H'
                LH_dict2[(userid, isbn)] = df_mat[k,2]


In [23]:
#print len(LH_dict)
print len(LH_dict2)
 

21313


In [24]:
dataframe_svd = pd.Series(LH_dict2).reset_index()   
dataframe_svd.columns = ["User-ID", "ISBN", "Book-Rating"]
dataframe_pivot_svd = dataframe_svd.pivot(index='User-ID', columns='ISBN', values='Book-Rating')

In [30]:
dataframe_svd_bi = pd.Series(LH_dict1).reset_index()   
dataframe_svd_bi.columns = ["User-ID", "ISBN", "Book-Rating"]

In [55]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

reader = Reader(rating_scale=(1,10))
Training_data = Dataset.load_from_df(dataframe_svd[["User-ID", "ISBN", "Book-Rating"]], reader)
Traindata, Testdata = train_test_split(Training_data, test_size=0.3)
algo = SVD()
predictions = algo.fit(Traindata).test(Testdata)
print(accuracy.rmse(predictions))

RMSE: 1.7088
1.7087661104727567


In [73]:
rmsenum = 0
rmseden = 0
truepos1 = 0
truepos0 = 0
falpos1 = 0
falpos0 = 0
trueneg1 = 0
trueneg0 = 0
falneg1 = 0
falneg0 = 0
total1 = 0
total0 = 0

for t in Testdata:
    testuid = t[0]
    testiid = t[1]
    #print t[2]," ",(algo.predict(iid=testiid,uid=testuid,r_ui=t[2]))[-2]
    if (algo.predict(iid=testiid,uid=testuid,r_ui=t[2]))[-2] > 7:
        predval = 1
    else: 
        predval = 0 
    if t[2] > 7: 
        trueval = 1
    else:
        trueval = 0
        
    if trueval == 1:
        total1 += 1
    else:
        total0 += 1
        
    if predval == trueval:
        if predval == 0:
            trueneg1 += 1
            truepos0 += 1
        else:
            truepos1 += 1
            trueneg0 += 1
        rmsenum += 1
    else:
        if predval == 0:
            falneg1 += 1
            falpos0 += 1
        else:
            falpos1 += 1
            falneg0 += 1
    rmseden += 1
    #print rmsenum," ",rmseden
rmse = (float)(float(rmsenum)/float(rmseden))
print "RMSE = ", rmse

RMSE =  0.643572098843


In [76]:
print truepos1," ", falpos1

3766   2080


In [74]:
precision1 = float(truepos1)/float(truepos1+falpos1)
recall1 = float(truepos1)/float(total1)
print "Precision for Love = ", precision1
print "Recall for Love = ", recall1

Precision for Love =  0.644201163189
Recall for Love =  0.949810844893


In [75]:
precision0 = float(truepos0)/float(truepos0+falpos0)
recall0 = float(truepos0)/float(total0)
print "Precision for Hate = ", precision0
print "Recall for Hate = ", recall0

Precision for Hate =  0.636861313869
Recall for Hate =  0.143680526966


In [9]:
unique, counts = numpy.unique(rating_matrix, return_counts=True)
dictwithcount = dict(zip(unique, counts))

In [48]:
print dictwithcount
totalL = 0
totalH = 0
for key in dictwithcount.keys():
    if key >0 and key <= 5:
        totalH = totalH + dictwithcount[key]
    elif key > 0 and key >= :
        totalL = totalL + dictwithcount[key]

{0: 2499842670, 1: 643, 2: 962, 3: 2085, 4: 3059, 5: 15978, 6: 13222, 7: 28226, 8: 39239, 9: 25888, 10: 28028}


In [49]:
print "L = ", totalL
print "H = ",totalH

L =  93155
H =  22727


In [33]:
print(rating_matrix.shape)

(50000L, 50000L)


In [34]:
import numpy as np
user_list = np.unique(df_mat[:,0])
print len(user_list)

105283


In [35]:
print len(bookmap.values())
print len(usermap.values())

#print df_mat[0,0]
#print bookmap[df_mat[0,1]]

50000
50000


In [36]:
import networkx as nx
from networkx.algorithms import bipartite

In [42]:
# constructing a bipartite graph

B = nx.Graph()
B.add_nodes_from(usermap.values(), bipartite=0) #0 indicates one class of nodes, here: users
B.add_nodes_from(bookmap.values(), bipartite=1) #1 for isbn

In [43]:
edge_ls = []

for k in range(0,800000):
    ls = []
    if k%50000==0:
        print k
    if df_mat[k,0] in usermap.keys() and df_mat[k,1] in bookmap.keys() and df_mat[k,2]!=0:
        ls.append(usermap[df_mat[k,0]])
        ls.append(bookmap[df_mat[k,1]])
        if df_mat[k,2] > 6:
            rate = 'L' # for love
        else: rate = 'H' # for hate
        ls.append(rate)
        edge_ls.append(ls)
#print edge_ls
B.add_weighted_edges_from(edge_ls)

'''
ls.append()
'''

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000


'\nls.append()\n'

In [57]:
print(len(B))

50000


In [ ]:
# import networkx as nx
# from networkx.algorithms import bipartite

# B = nx.Graph()
# B.add_nodes_from(usermap.values(), bipartite=0)
# B.add_nodes_from(bookmap.values(), bipartite=1)

# edge_ls = []
# for i in range(0,50000):
#     for j in range(0,50000):
#         if rating_matrix[i,j] !=0:
#             ls = []
#             ls.append(i)
#             ls.append(j)
#             if rating_matrix[i,j] > 6:
#                 rate = 'L' # for love
#             else: 
#                 rate = 'H' # for hate
#             ls.append(rate)
#             edge_ls.append(ls)
# B.add_weighted_edges_from(edge_ls)

In [1]:
%store -r B
import pickle
pickle.dump(B,open("a.p","wb"))

In [47]:
import pickle
#B= pickle.load(open("a.p","rb"))
from collections import defaultdict
Dictionary = defaultdict(dict)
count =0
dictionary_adj = dict(B.adjacency()) 
for user1 in usermap.values():
    BFS_Dict= dict(nx.bfs_successors(B,user1))
    User_item_list = []
    User_item_list.append(user1)
    edge_list=[]
    if count%10000 ==0:
        print "Reached user ",count+1
    count+=1
    if bool(BFS_Dict):
        for book1 in BFS_Dict[user1]:
            if book1 in BFS_Dict.keys():
                edge_list.append(dictionary_adj[user1][book1]['weight'])
                for user2 in BFS_Dict[book1]:
                    edge_list.append(dictionary_adj[book1][user2]['weight'])    
                    '''
                    if count%100 ==0:
                        print "Hello"
                        print dictionary_adj[user2]
                        print BFS_Dict[user1]
                    '''
                    for book2 in dictionary_adj[user2].keys(): 
                        if book2 in BFS_Dict[user1] and book2!=book1:
                            edge_list.append(dictionary_adj[user2][book2]['weight'])
                            User_item_list.append(book2)
                            Dictionary[tuple(User_item_list)][tuple(edge_list)] = Dictionary[tuple(User_item_list)].get(tuple(edge_list),0)+1
                            Dictionary[tuple(User_item_list)]["Rating"]=rating_matrix[User_item_list[0]][User_item_list[1]]
                            edge_list.pop()
                            User_item_list.pop()
                    edge_list.pop()
                edge_list.pop()
            
#print(Dictionary)
                        

Reached user  1
Reached user  10001
Reached user  20001
Reached user  30001
Reached user  40001


In [58]:
print (len(Dictionary))

161293


In [48]:
%store Dictionary
import pickle
pickle.dump(Dictionary,open("b.p","wb"))

Stored 'Dictionary' (defaultdict)


In [5]:
# naming all configurations
con = []

con.append([('H', 'H', 'H')])
con.append([('H', 'H', 'L')])
con.append([('H', 'L', 'L')])
con.append([('L', 'L', 'L')])
con.append([('L', 'L', 'H')])
con.append([('L', 'H', 'H')])
con.append([('H', 'L', 'H')])
con.append([('L', 'L', 'L')])



In [6]:
%store -r Dictionary
import numpy as np
config_mat = np.zeros((len(Dictionary), 9))
pair_dic = {}
i=0
for k in Dictionary.keys():
    userItemConfig = Dictionary[k]
    pair_dic[i] = k
    for key in userItemConfig.keys():
        for j in range(0,8):
            if con[j][0] == key:
                config_mat[i,j] = userItemConfig[key]
    if rating_matrix[k[0],k[1]]!=0:
        if rating_matrix[k[0],k[1]] > 6:
            config_mat[i,-1] = 1
        else: config_mat[i,-1] = 0
    #config_mat[i,-1] = rating_matrix[k[0],k[1]] 
    i+=1
            

In [59]:
print(len(config_mat))

161293


In [51]:
%store config_mat
import pickle
pickle.dump(config_mat,open("c.p","wb"))

Stored 'config_mat' (ndarray)


In [164]:
%store -r config_mat
print(config_mat)

[[ 0.  0.  0. ...  0.  1.  1.]
 [ 0.  0.  0. ...  0.  0.  1.]
 [ 0.  0.  0. ...  0.  1.  0.]
 ...
 [ 0.  1. 10. ...  1.  8.  0.]
 [ 0.  0.  0. ...  1.  3.  0.]
 [ 0.  1.  2. ...  0.  0.  0.]]


In [168]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(config_mat[:,0:8], config_mat[:,-1], test_size=0.30, shuffle=False)

from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [171]:
mnb_y_pred = mnb.predict(X_test)
print len(mnb_y_pred)

48388


In [172]:
#print len(y_test)

from sklearn.metrics import mean_squared_error
from math import sqrt

rmse = sqrt(mean_squared_error(y_test, mnb_y_pred))
print "RMSE with Naive Bayes:",rmse

RMSE with Naive Bayes: 0.66533580754


In [60]:
%store X_train
%store X_test
%store y_train
%store y_test

Stored 'X_train' (ndarray)
Stored 'X_test' (ndarray)
Stored 'y_train' (ndarray)
Stored 'y_test' (ndarray)


In [173]:
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from math import sqrt

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
KNN_y_pred = knn.predict(X_test)
rmse_knn = sqrt(mean_squared_error(y_test, KNN_y_pred))
print "RMSE with kNN:", rmse_knn

RMSE with kNN: 0.665118341998


In [227]:
%store df_mat_book
%store bookmap
%store usermap
%store pair_dic
%store lr_y_pred

Stored 'df_mat_book' (ndarray)
Stored 'bookmap' (dict)
Stored 'usermap' (dict)
Stored 'pair_dic' (dict)
Stored 'lr_y_pred' (ndarray)


In [25]:
# Demo
%store -r X_train
%store -r y_test
%store -r lr_y_pred
%store -r df_mat_book
%store -r usermap
%store -r bookmap
%store -r pair_dic


from random import randint
isbn =[row[0] for row in df_mat_book]
book_name =[row[1] for row in df_mat_book]
count=0
# 5922 , 23938 
offset_train = len(X_train)
offset_test = len(y_test)
while(count<5):
    rnd = randint(0,offset_test-1)
    #print pair_dic[rnd][0]
    if lr_y_pred[rnd] == 0:
        lr_p = 'Hate'
    else : lr_p = 'Love'
    if y_test[rnd] == 0:
        yt_p = 'Hate'
    else : yt_p = 'Love'   
    if isbn.count(bookmap.keys()[bookmap.values().index(pair_dic[rnd+offset_train][1])])!=0:
        print " USER ID: ",usermap.keys()[usermap.values().index(pair_dic[rnd+offset_train][0])],
        print " BOOK NAME: ",book_name[isbn.index(bookmap.keys()[bookmap.values().index(pair_dic[rnd+offset_train][1])])],
        print " PREDICTED: ",lr_p," ACTUAL: ",yt_p
        count+=1


 USER ID:  79278  BOOK NAME:  The Toilet Paper Tigers  PREDICTED:  Love  ACTUAL:  Love
 USER ID:  16795  BOOK NAME:  The Diving Bell and the Butterfly : A Memoir of Life in Death  PREDICTED:  Hate  ACTUAL:  Love
 USER ID:  179716  BOOK NAME:  The Drawing of the Three (The Dark Tower, Book 2)  PREDICTED:  Hate  ACTUAL:  Hate
 USER ID:  135143  BOOK NAME:  Defy Not The Heart  PREDICTED:  Hate  ACTUAL:  Hate
 USER ID:  19707  BOOK NAME:  The Color of Water: A Black Man's Tribute to His White Mother  PREDICTED:  Hate  ACTUAL:  Hate


In [127]:
# import pandas as pd
# df_book = pd.read_csv('BX-Books.csv', names=["ISBN","Book-Title","Book-Author","Year-Of-Publication","Publisher","Image-URL-S","Image-URL-M","Image-URL-L"], delimiter=";")
# df_mat_book = df_book.as_matrix()[1:]

In [200]:
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test

from sklearn.svm import LinearSVC
from sklearn.metrics import mean_squared_error
from math import sqrt

clf = LinearSVC()

clf = clf.fit(X_train, y_train)
svc_y_pred = clf.predict(X_test)
rmse_svc = sqrt(mean_squared_error(y_test, svc_y_pred))
print "RMSE with SVM:", rmse_svc

RMSE with SVM: 0.629274668368


In [201]:
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test

from sklearn import tree
from sklearn.metrics import mean_squared_error
from math import sqrt

dt = tree.DecisionTreeClassifier()
dt = dt.fit(X_train, y_train)
dt_y_pred = dt.predict(X_test)
rmse_dt = sqrt(mean_squared_error(y_test, dt_y_pred))
print "RMSE with Decision tree:", rmse_dt

RMSE with Decision tree: 0.633529668826


In [202]:
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test

from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from math import sqrt

lr = linear_model.LogisticRegression()
lr = lr.fit(X_train, y_train)
lr_y_pred = lr.predict(X_test)
rmse_lr = sqrt(mean_squared_error(y_test, lr_y_pred))
print "RMSE with Logistic Regression:", rmse_lr

RMSE with Logistic Regression: 0.628995454256
